In [1]:
from collections import defaultdict
from itertools import product

We are going to give names to the spaces:

```text
#############
#abcdefghijk#
###l#m#n#o###
  #p#q#r#s#
  #########
```

In [2]:
neighbours: dict[str, tuple[str]] = {
    'a': ('b'),
    'b': ('a', 'c'),
    'c': ('b', 'd', 'l'),
    'd': ('c', 'e'),
    'e': ('d', 'f', 'm'),
    'f': ('e', 'g'),
    'g': ('f', 'h', 'n'),
    'h': ('g', 'i'),
    'i': ('h', 'j', 'o'),
    'j': ('i', 'k'),
    'k': ('j',),
    'l': ('c', 'p'),
    'm': ('e', 'q'),
    'n': ('g', 'r'),
    'o': ('i', 's'),
    'p': ('l',),
    'q': ('m',),
    'r': ('n',),
    's': ('o'),
}

In [3]:
def check_neighbours():
    for s, n in neighbours.items():
        for ns in n:
            if not s in neighbours[ns]:
                print(f"Space {s} not listed in {ns}")

In [4]:
rooms: dict[str, tuple[str]] = {
    'A': ('l', 'p'),
    'B': ('m', 'q'),
    'C': ('n', 'r'),
    'D': ('o', 's'),
}

inner_rooms = {k: v[1] for k, v in rooms.items()}
outer_rooms = {k: v[0] for k, v in rooms.items()}

outsides = tuple('cegi')
hallway = tuple(chr(i+97) for i in range(ord('l')-97))

In [5]:
energy = {
    'A': 1,
    'B': 10,
    'C': 100,
    'D': 1000,
}

In [83]:
class Map:
    def __init__(self, amphipods: dict[str, str]):
        self.amphipods = amphipods
        self.keys = ["".join(p) for p in product('ABCD', '01')]

        self.occupation = defaultdict(lambda: '.')
        for amphi, p in self.amphipods.items():
            self.occupation[p] = amphi[0]
        self.last_moved = ''

    def __str__(self) -> str:
        m = "#############\n#"
        for c in range(ord('a'), ord('l')):
            m += self.occupation[chr(c)]
        m += f"#\n###{self.occupation['l']}#{self.occupation['m']}#{self.occupation['n']}#{self.occupation['o']}###\n"
        m += f"  #{self.occupation['p']}#{self.occupation['q']}#{self.occupation['r']}#{self.occupation['s']}#  \n"
        m += "  #########"
        return m

    @classmethod
    def from_map(cls, room_map: str) -> "Map":
        room = 0
        pos = {'A': [], 'B': [], 'C': [], 'D': []}
        for l in room_map.split('\n'):
            for c in l:
                match c:
                    case '.':
                        room += 1
                    case '#' | ' ':
                        pass
                    case str(a):
                        pos[a].append(chr(97+room))
                        room += 1
        amphipods = {k+str(i): v[i] for k, v in pos.items() for i in range(2)}
        return cls(amphipods)

    def code(self) -> str:
        return "".join(self.amphipods[k] for k in self.keys)

    def move(self, amphi: str, dest: str):
        if self.occupation[dest] != '.':
            return None
        if self.amphipods[amphi] == dest:
            return None
        new_amphis = {k: v for k, v in self.amphipods.items()}
        new_amphis[amphi] = dest
        m = Map(new_amphis)
        m.last_moved = amphi
        return m

    def is_in_room(self, amphi: str) -> bool:
        return self.amphipods[amphi] in rooms[amphi[0]]

    def all_in_rooms(self) -> bool:
        return all(self.is_in_room(a) for a in self.amphipods.keys())

    def is_blocking_inner(self, amphi: str) -> bool:
        return (self.amphipods[amphi] in outer_rooms[amphi[0]]) and (self.occupation[inner_rooms[amphi[0]]] != amphi[0])

    def is_outside(self, amphi: str) -> bool:
        return self.amphipods[amphi] in outsides

    def is_in_hallway(self, amphi: str) -> bool:
        return self.amphipods[amphi] in hallway

    def is_empty(self, r: str) -> bool:
        return self.occupation[r] == '.'

    def can_go_room(self, amphi: str) -> bool:
        return (self.occupation[outer_rooms[amphi[0]]] == '.') and (self.occupation[inner_rooms[amphi[0]]] in ('.', amphi[0]))

    def propose_moves(self, amphi: str) -> list[str]:
        pos = self.amphipods[amphi]
        if self.is_in_room(amphi) and not(self.is_blocking_inner(amphi)):
            return []
        if (pos == outer_rooms[amphi[0]]) and self.is_empty(inner_rooms[amphi[0]]):
                return [inner_rooms[amphi[0]],]
        l = []
        for r in neighbours[pos]:
                if self.is_empty(r):
                    if r in outer_rooms.values():
                        if pos in inner_rooms.values():
                            l.append(r)
                        if r == outer_rooms[amphi[0]] and self.occupation[inner_rooms[amphi[0]]] in ('.', amphi[0]):
                            l.append(r)
                    else:
                        l.append(r)
        return l

    def next_moves(self) -> dict[str, list[str]]:
        moves = dict()
        if self.last_moved and self.amphipods[self.last_moved] in outsides:
            return {self.last_moved: self.propose_moves(self.last_moved)}
        for amphi in self.amphipods.keys():
            if (amphi == self.last_moved) or not(self.is_in_hallway(amphi)) or self.can_go_room(amphi):
                m = self.propose_moves(amphi)
                if m != []:
                    moves.update({amphi: m})
        return moves

In [7]:
test = """
#############
#...........#
###B#C#B#D###
  #A#D#C#A#
  #########
"""

In [84]:
map_test = Map.from_map(test)
print(map_test)
map_test.next_moves()

#############
#...........#
###B#C#B#D###
  #A#D#C#A#  
  #########


{'B0': ['c'], 'B1': ['g'], 'C0': ['e'], 'D0': ['i']}

In [87]:
def energy_path(map_input: Map) -> int:
    maps = [(map_input, 0),]
    dict_maps: dict[str, int] = dict()
    min_energy = 1000000000000

    while len(maps) > 0:
        next_maps = []
        for m, en_m in maps:
            for amphi, spaces in m.next_moves().items():
                en = energy[amphi[0]] + en_m
                for s in spaces:
                    new_map = m.move(amphi, s)
                    if (code := new_map.code()) in dict_maps.keys():
                        if en < dict_maps[code]:
                            dict_maps[code] = en
                            next_maps.append((new_map, en))
                            if new_map.all_in_rooms():
                                min_energy = min(min_energy, en)
                    else:
                        dict_maps[code] = en
                        next_maps.append((new_map, en))
                        if new_map.all_in_rooms():
                            min_energy = min(min_energy, en)
        maps = next_maps
    return min_energy


In [88]:
energy_path(map_test)

12521

In [89]:
inp = """
#############
#...........#
###D#C#D#B###
  #C#A#A#B#
  #########
"""

In [90]:
energy_path(Map.from_map(inp))

14415